In [1]:
import numpy as np 
import scipy as sp

In [2]:
import sys
quic_path = '/home/quic/QUIC-Projects'
if not quic_path in sys.path:
    sys.path.append(quic_path)

In [3]:
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit import QuantumCircuit
qc = PauliFeatureMap(feature_dimension=2, reps=2)
qc.draw()

┌───┐┌─────────────┐                                          ┌───┐»
q_0: ┤ H ├┤ P(2.0*x[0]) ├──■────────────────────────────────────■──┤ H ├»
     ├───┤├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐├───┤»
q_1: ┤ H ├┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├┤ H ├»
     └───┘└─────────────┘└───┘└──────────────────────────────┘└───┘└───┘»
«     ┌─────────────┐                                          
«q_0: ┤ P(2.0*x[0]) ├──■────────────────────────────────────■──
«     ├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐
«q_1: ┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├
«     └─────────────┘└───┘└──────────────────────────────┘└───┘

In [4]:
qc.bind_parameters({x:1 for x in list(qc.parameters)}).draw()

┌───┐┌─────────────────────┐                                 ┌───┐»
q_0: ┤ H ├┤ P(2.00000000000000) ├──■───────────────────────────■──┤ H ├»
     ├───┤├─────────────────────┤┌─┴─┐┌─────────────────────┐┌─┴─┐├───┤»
q_1: ┤ H ├┤ P(2.00000000000000) ├┤ X ├┤ P(9.17283818781954) ├┤ X ├┤ H ├»
     └───┘└─────────────────────┘└───┘└─────────────────────┘└───┘└───┘»
«     ┌─────────────────────┐                                 
«q_0: ┤ P(2.00000000000000) ├──■───────────────────────────■──
«     ├─────────────────────┤┌─┴─┐┌─────────────────────┐┌─┴─┐
«q_1: ┤ P(2.00000000000000) ├┤ X ├┤ P(9.17283818781954) ├┤ X ├
«     └─────────────────────┘└───┘└─────────────────────┘└───┘

In [5]:
G = PauliFeatureMap(feature_dimension=2, reps=2, parameter_prefix='x').to_instruction()
Gdg = PauliFeatureMap(feature_dimension=2, reps=2, parameter_prefix='y').to_instruction().inverse()
circ = QuantumCircuit(2)
circ.append(G, (0, 1))
circ.append(Gdg, qargs=(0,1))
circ.draw()
circ.parameters

{Parameter(x[0]), Parameter(x[1]), Parameter(y[0]), Parameter(y[1])}

In [6]:
qc = PauliFeatureMap(feature_dimension=2, reps=2)
qc = qc.assign_parameters({list(qc.parameters)[i]:1 for i in range(2)})
from qiskit import execute
from qiskit.providers.aer import StatevectorSimulator
vec = execute(qc, StatevectorSimulator()).result().get_statevector()
np.abs(np.vdot(vec, vec))**2

1.0000000000000009

In [7]:
import numpy as np
a = np.array([1, 1j])
np.vdot(a, a)

(2+0j)

In [8]:
from itertools import product
a = list(product((0,1), repeat=3))
''.join(map(str, a[0]))

'000'

In [9]:
def _process_counts(n, dic, *count):
    temp = 0
    for bin in product((0, 1), repeat=n):
        val1 = (-1)**sum([bin[c] for c in count])
        val2 = dic.get(''.join(map(str, bin)), 0)
        temp += val1*val2
    return temp

In [10]:
import numpy as np 
np.random.seed(10)
dic = {k:np.random.randint(0, 100) for k in map(''.join, product('01', repeat=3))}
_process_counts(3, dic, 0)

-103

In [11]:
from qiskit.circuit import QuantumRegister
list(QuantumRegister(4))

[Qubit(QuantumRegister(4, 'q3'), 0),
 Qubit(QuantumRegister(4, 'q3'), 1),
 Qubit(QuantumRegister(4, 'q3'), 2),
 Qubit(QuantumRegister(4, 'q3'), 3)]

In [12]:
from qiskit.aqua.components.feature_maps import FeatureMap
from qiskit.circuit.library import ZZFeatureMap
issubclass(FeatureMap, QuantumCircuit)

False

In [13]:
from qiskit.circuit import Parameter
qc = QuantumCircuit(4)
#qc.rx(Parameter('t'), 0)
qc.to_gate().params

[]

In [14]:
from qiskit.circuit.library import TwoLocal
TwoLocal(1, ['rx', 'rz'], reps=1, skip_final_rotation_layer=True).draw()

┌──────────┐┌──────────┐
q_0: ┤ RX(θ[0]) ├┤ RZ(θ[1]) ├
     └──────────┘└──────────┘

In [15]:
list(QuantumRegister(1))+list(QuantumRegister(2))

[Qubit(QuantumRegister(1, 'q4'), 0),
 Qubit(QuantumRegister(2, 'q5'), 0),
 Qubit(QuantumRegister(2, 'q5'), 1)]

In [21]:
from qiskit.providers.aer import UnitarySimulator, QasmSimulator
qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(1,2)
qc.draw()

┌───┐          
q_0: ┤ H ├──■───────
     └───┘┌─┴─┐     
q_1: ─────┤ X ├──■──
          └───┘┌─┴─┐
q_2: ──────────┤ X ├
               └───┘

In [26]:
job = execute(qc, backend=UnitarySimulator())

In [34]:
qc.ucrx([0, 1, 2, 3], [0,1], 2)
qc.draw()

┌───┐          ┌────────────────┐┌────────────────┐┌────────────────┐
q_0: ┤ H ├──■───────┤1               ├┤1               ├┤1               ├
     └───┘┌─┴─┐     │                ││                ││                │
q_1: ─────┤ X ├──■──┤2 UCRX(0,1,2,3) ├┤2 UCRX(0,1,2,3) ├┤2 UCRX(0,1,2,3) ├
          └───┘┌─┴─┐│                ││                ││                │
q_2: ──────────┤ X ├┤0               ├┤0               ├┤0               ├
               └───┘└────────────────┘└────────────────┘└────────────────┘

In [133]:
from qiskit.circuit.library import U3Gate, RXGate, XGate
n = 2
qc = QuantumCircuit(n+2)
qc.compose(TwoLocal(n, ['ry', 'rz'], 'cz'), inplace=True)
qc.uc([U3Gate(theta, phi, 0).to_matrix() for theta, phi in np.random.rand(int(2**n), 2)], list(range(n)),n)
#qc.ucry(list(np.pi*np.random.rand(int(2**(n-1)))), list(range(n-1)), n-1)
#qc.ucrz(list(2*np.pi*np.random.rand(int(2**(n-1)))), list(range(n-1)), n-1)
qc.ucrx(list(np.pi*np.random.rand(int(2**n))), list(range(n)), n+1)
qc.cswap(0, n, n+1)
qc.draw()


┌──────────┐┌──────────┐   ┌──────────┐┌──────────┐   ┌──────────┐»
q_0: ┤ RY(θ[0]) ├┤ RZ(θ[2]) ├─■─┤ RY(θ[4]) ├┤ RZ(θ[6]) ├─■─┤ RY(θ[8]) ├»
     ├──────────┤├──────────┤ │ ├──────────┤├──────────┤ │ ├──────────┤»
q_1: ┤ RY(θ[1]) ├┤ RZ(θ[3]) ├─■─┤ RY(θ[5]) ├┤ RZ(θ[7]) ├─■─┤ RY(θ[9]) ├»
     └──────────┘└──────────┘   └──────────┘└──────────┘   └──────────┘»
q_2: ──────────────────────────────────────────────────────────────────»
                                                                       »
q_3: ──────────────────────────────────────────────────────────────────»
                                                                       »
«     ┌───────────┐   ┌───────────┐┌───────────┐┌──────────────┐»
«q_0: ┤ RZ(θ[10]) ├─■─┤ RY(θ[12]) ├┤ RZ(θ[14]) ├┤1             ├»
«     ├───────────┤ │ ├───────────┤├───────────┤│              │»
«q_1: ┤ RZ(θ[11]) ├─■─┤ RY(θ[13]) ├┤ RZ(θ[15]) ├┤2 MULTIPLEXER ├»
«     └───────────┘   └───────────┘└───────────┘│              │»
«q_2: ──────────────────────────────────────────┤0             ├»
«                                               └──────────────┘»
«q_3: ──────────────────────────────────────────────────────────»
«                                                               »
«     ┌──────────────────────────────────────┐   
«q_0: ┤1                                     ├─■─
«     │                                      │ │ 
«q_1: ┤2                                     ├─┼─
«     │  UCRX(2.6848,1.6574,2.6137,0.023799) │ │ 
«q_2: ┤                                      ├─X─
«     │                                      │ │ 
«q_3: ┤0                                     ├─X─
«     └──────────────────────────────────────┘

In [134]:
from qiskit import transpile
transpile(qc, basis_gates=['cx', 'rx', 'ry', 'rz'], optimization_level=3).depth()

100